this is the second part, and it uses files from first run of centrostruct.py module


as result we should have tab with line azimuths, line angles, structure azimuth and rotations. 


i am also going to add xarms here - sizes and angles

In [14]:
from pathlib import Path
import pandas as pd
import numpy as np
from shapely.geometry import Point, MultiPoint, LineString, MultiLineString
import csv
from math import sqrt, acos, degrees

In [2]:
# пути
workdir = Path(r'D:\work\2022_m\training\ok')
resultdir = workdir / 'result'
tempdir = workdir / 'temp'

some functions for calc

In [12]:
# from math import sqrt, acos, degrees

def azimuth(a, b):
    # by two points we get azimuth
    dx = a[0] - b[0]
    dy = a[1] - b[1]
    dist = round(sqrt(dx*dx + dy*dy), 2)    # dist a to b
    dx2 = abs(dx)
    beta = degrees(acos(dx2/dist))
    if dx > 0:
        if dy < 0:
            angle = 270 + beta
        else:
            angle = 270 - beta
    else:
        if dy < 0:
            angle = 90 - beta
        else:
            angle = 90 + beta

    return round(angle, 1), dist

In [15]:
# import csv

def xyz_read(path):
    # read xyz arrays from files
    with open(path, newline='') as xyz_file:
        spamlist = []
        spamreader = csv.reader(xyz_file, delimiter=' ', quoting=csv.QUOTE_NONNUMERIC, skipinitialspace=True)
        for row in spamreader:
            spamlist.append(row)

    return spamlist

In [105]:
def xarms_levels(array, segment_size):
    segmented = []
    start = min(array)
    end = start + segment_size
    while end < max(array):
        segmented.append(sum(end > i > start for i in np_str_z))
        start += segment_size
        end += segment_size

        
    # then choose only bigget than mean*2
    ma = []
    for nu, i in enumerate(segmented):
        if i > (np.mean(segmented)*2):
            ma.append((nu, i))

            
    # then group them if they close to each other
    ma2 = {}
    low_level = 0
    points_sum = 0

    for i in range(len(ma)):
        # если следующий сегмент также отобран
        if i != len(ma)-1 and ma[i][0]+1 == (ma[i+1][0]):
            points_sum += ma[i+1][1]

            # если предыдущий сегмент не был отобран 
            if i > 0 and not ma[i][0]-1 == (ma[i-1][0]):
                points_sum += ma[i][1]
                low_level = ma[i][0]

            ma2[low_level] = points_sum

        else:
            #print('skip')
            low_level = 0
            points_sum = 0

            
    #  из отобранных групп выделяем три наибольшие
    # такой отбор не стаботает на траверсах постоянной высоты - там будет по два выброса на траверсу
    selected = sorted(ma2.items(), key=lambda item: item[1])[-3:]
    #print(selected)

    h_trav = []   # нижие кромки траверс
    for i in selected:
        h_trav.append(round((min(array) + segment_size*i[0]), 1))
    #print(sorted(h_trav))    
    
    return sorted(h_trav)


In [114]:
def midpoint(p1, p2):
    # calc middle point
    mx = round((p1[0]+p2[0])/2, 2)
    my = round((p1[1]+p2[1])/2, 2)
    
    return (mx, my)


firstly get the initial table 

In [5]:
cgt_tab = pd.read_excel(resultdir / 'cgtw_output.xlsx', index_col=0)

In [6]:
cgt_tab[:3]

,c_line,id,x,y,z,geometry,havepoints,x1,y1,z1,x1_t,y1_t,z1_t,angle
index,,,,,,,,,,,,,,
1,1,1,1512936.51,5365808.23,107.81,POINT Z (1512936.51 5365808.23 107.81),1,1512936.52,5365808.24,76.44,1512936.52,5365808.22,107.81,0
2,1,2,1512632.98,5365943.75,99.71,POINT Z (1512632.98 5365943.75 99.71),1,1512633.05,5365943.64,69.67,1512633.02,5365943.77,99.71,0
3,1,3,1512336.16,5366075.89,106.28,POINT Z (1512336.16 5366075.89 106.28),1,1512336.18,5366075.94,70.94,1512336.02,5366076.17,106.28,0


create new tab based of initial

In [7]:
ang_tab = cgt_tab[['c_line', 'id']].copy()

In [8]:
# add new cols
new_cols = ['span name', 'span length 2d', 'span azimuth', 'line angle', 'structure azimuth', 'structure rotation', 'x_arm start']
for col in new_cols:
    ang_tab[col] = np.nan

In [9]:
ang_tab[:3]

,c_line,id,span name,span length 2d,span azimuth,line angle,structure azimuth,structure rotation,x_arm start
index,,,,,,,,,
1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN


find span name, length, azimuth, line angle

In [10]:
for n in range(len(cgt_tab)-1):
    idx = cgt_tab.iloc[n].name  # find index and work with index
    
    if cgt_tab.loc[idx, 'c_line'] == cgt_tab.loc[idx+1, 'c_line']:
        # check if we on the same c-line
        ang_tab.loc[idx, 'span name'] = f"{cgt_tab.loc[idx, 'id']} - {cgt_tab.loc[idx+1, 'id']}"  # span id 
        a = (cgt_tab.loc[idx, 'x1'], cgt_tab.loc[idx, 'y1'])   # start point
        b = (cgt_tab.loc[idx+1, 'x1'], cgt_tab.loc[idx+1, 'y1'])   # end point
        ang_tab.loc[idx, 'span azimuth'], ang_tab.loc[idx, 'span length 2d'] = azimuth(a, b)
        if idx > 1:
            if cgt_tab.loc[idx, 'c_line'] == cgt_tab.loc[idx-1, 'c_line']:
                spam = ang_tab.loc[idx, 'span azimuth'] - ang_tab.loc[idx-1, 'span azimuth']
                if spam < -180:
                    ang_tab.loc[idx, 'line angle'] = spam + 360
                else:
                    ang_tab.loc[idx, 'line angle'] = spam
            else:
                ang_tab.loc[idx, 'line angle'] = 0
        else:
                ang_tab.loc[idx, 'line angle'] = 0
    
    #print(span_name, azim, span_len)

In [11]:
ang_tab[-3:]

,c_line,id,span name,span length 2d,span azimuth,line angle,structure azimuth,structure rotation,x_arm start
index,,,,,,,,,
23,1,23,23 - 24,206.83,13.4,-0.1,NaN,NaN,NaN
24,1,24,24 - 25,241.15,13.5,0.1,NaN,NaN,NaN
25,1,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN


lets work with x-arms

firstly  will find the start level - the lowest xarm on structure

for this work we need to upload ALS structure points - we will use created on the first step

In [16]:
# we should go for initial table and select each str again

# here in notepad we will work with first structure then all the script will work as separate py file
#for n in range(len(cgt_tab)):
#    idx = cgt_tab.iloc[n].name  # find index and work with index
#    tow_cut = xyz_read(tempdir / str(f"{idx}_{cgt_tab.loc[idx, 'id']}_str.xyz"))

idx = 1    # work with one structure only now
tow_cut = xyz_read(tempdir / str(f"{idx}_{cgt_tab.loc[idx, 'id']}_str.xyz"))

In [110]:
np_str = np.array(tow_cut)
np_str_z = np_str[:,2]    # leave only z-coord
str_hight = round(max(np_str_z)-min(np_str_z), 2)
np_str_z = np.delete(np_str_z, np_str_z<str_hight+4, 0)   # cut 4 m from grd

In [108]:
[z1, z2, z3] = xarms_levels(np_str_z, 0.1)   # find xarm levels
z1

87.0

In [112]:
# cut the legs again (more accurate)
np_str_cut = np.delete(np_str, np_str[:,2]<(z1-0.1), 0)

# now work with top part
np_str_mp = MultiPoint(np_str_cut)    # to multipoint (needs to shapely)
rect = np_str_mp.minimum_rotated_rectangle
rect_c = rect.centroid

In [115]:
# find a longest side and azimuth

side_a = azimuth(list(rect.exterior.coords)[0], list(rect.exterior.coords)[1])
side_b = azimuth(list(rect.exterior.coords)[1], list(rect.exterior.coords)[2])
if side_a[1] > side_b[1]:
    str_azmt = side_a[0]
    axl = LineString([midpoint(list(rect.exterior.coords)[0], list(rect.exterior.coords)[3]), 
                     midpoint(list(rect.exterior.coords)[1], list(rect.exterior.coords)[2])])
else:
    str_azmt = side_b[0]
    axl = LineString([midpoint(list(rect.exterior.coords)[0], list(rect.exterior.coords)[1]), 
                     midpoint(list(rect.exterior.coords)[2], list(rect.exterior.coords)[3])])


# def midpoint(p1, p2):
#     # calc middle point
#     mx = round((p1[0]+p2[0])/2, 2)
#     my = round((p1[1]+p2[1])/2, 2)
    
#     return (mx, my)

print(axl.length)
print(axl)
print(str_azmt)

7.394166619878231
LINESTRING (1512934.52 5365805.16, 1512938.61 5365811.32)
(33.6, 7.4)
(303.7, 4.16)
33.6


In [128]:
axl.coords[1][0]


1512938.61

In [ ]:
94 98 101